In [ ]:
!pip install gradio scikit-learn joblib pandas matplotlib seaborn wordcloud flask flask-cors pyngrok

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import re
import gradio as gr
from datetime import datetime
from collections import Counter
from wordcloud import WordCloud

In [ ]:
# Dataset
# Enfoque específico en tiendas virtuales: Aliexpress, Amazon, Ebay, Temu

emails = [
    # SPAM (60% = 24 correos) - Enfoque en tiendas virtuales fraudulentas
    "¡OFERTA EXCLUSIVA! Compra ahora en AliExpress con 90% de descuento. Solo hoy!",
    "Amazon te ha seleccionado para una oferta especial. Reclama tu premio de $1000",
    "eBay: Tu cuenta será suspendida si no verificas tu información bancaria AHORA",
    "TEMU FLASH SALE: iPhone 15 por solo $50. Oferta limitada, compra ya!",
    "AliExpress Black Friday: Productos gratis, solo paga envío. Haz clic aquí",
    "Amazon Prime: Has ganado una membresía gratuita de por vida. Confirma aquí",
    "eBay URGENTE: Problema con tu cuenta. Verifica o será cerrada permanentemente",
    "TEMU MEGA OFERTA: Laptop gaming por $99. Solo quedan 3 unidades!",
    "AliExpress te regala $500 en cupones. No pierdas esta oportunidad única",
    "Amazon: Reembolso pendiente de $350. Haz clic para reclamar inmediatamente",
    "eBay: Felicidades, has sido seleccionado para nuestro programa VIP exclusivo",
    "TEMU LIQUIDACIÓN: Productos de marca a precios increíbles. Compra antes que se agoten",
    "¡GANASTE! AliExpress sorteo: iPhone, iPad y $1000 en efectivo. Reclama ya",
    "Amazon Security Alert: Actividad sospechosa detectada. Verifica tu cuenta",
    "eBay: Tu artículo favorito tiene 95% de descuento. Oferta expira en 1 hora",
    "TEMU EXCLUSIVO: Productos premium gratis, solo paga $1 de envío",
    "AliExpress VIP: Acceso exclusivo a productos con descuentos del 99%",
    "Amazon: Tu paquete está retenido. Paga $5 para liberarlo inmediatamente",
    "eBay FINAL SALE: Electrónicos de marca a precios de liquidación total",
    "TEMU ALERT: Tu carrito tiene productos reservados. Compra en 10 minutos",
    "¡ÚLTIMO DÍA! AliExpress cierra con descuentos históricos del 95%",
    "Amazon Prime Day secreto: Ofertas exclusivas solo para ti. Accede ahora",
    "eBay: Problema de pago detectado. Actualiza tu método de pago urgentemente",
    "TEMU FINAL HOURS: Productos gratis, aprovecha antes del cierre definitivo",
    
    # NO SPAM (40% = 16 correos) - Comunicaciones legítimas
    "Hola María, ¿cómo estás? Espero que tengas un buen día",
    "Recordatorio: Reunión de equipo mañana a las 10:00 AM en la sala de conferencias",
    "Tu pedido de Amazon ha sido enviado. Número de seguimiento: 1Z999AA1234567890",
    "eBay: Confirmación de compra. Gracias por tu pedido #EB123456789",
    "Informe mensual de ventas adjunto. Favor revisar antes de la reunión",
    "¿Te gustaría almorzar el viernes? Conozco un restaurante nuevo muy bueno",
    "AliExpress: Tu pedido ha llegado al centro de distribución local",
    "Feliz cumpleaños! Espero que tengas un día maravilloso con tu familia",
    "TEMU: Confirmación de entrega. Tu paquete fue entregado exitosamente",
    "Propuesta de proyecto adjunta. Por favor revisa y envía tus comentarios",
    "Amazon: Tu suscripción a Prime se renovará automáticamente el próximo mes",
    "Invitación a la conferencia de tecnología del próximo trimestre",
    "eBay: Califica tu experiencia de compra reciente. Tu opinión es importante",
    "Actualización del sistema programada para el fin de semana. Planifica en consecuencia",
    "TEMU: Tu reseña del producto ha sido publicada. Gracias por compartir",
    "Resumen semanal del proyecto. Todos los hitos se completaron a tiempo"
]

# Etiquetas correspondientes (1 = spam, 0 = no spam)
labels = [1]*24 + [0]*16  # 60% spam, 40% no spam

print(f"Total de emails: {len(emails)}")
print(f"Emails spam: {sum(labels)} ({sum(labels)/len(labels)*100:.1f}%)")
print(f"Emails no spam: {len(labels)-sum(labels)} ({(len(labels)-sum(labels))/len(labels)*100:.1f}%)")

In [ ]:
print(labels)

In [ ]:
df = pd.DataFrame({
    'email': emails,
    'label': labels,
    'category': ['Spam' if label == 1 else 'No Spam' for label in labels]
})

# Calcular longitudes
df['length'] = df['email'].str.len()

# 1. Gráfico de longitud de emails (alternativa al boxplot)
plt.figure(figsize=(15, 12))

# Subplot 1: Histograma comparativo de longitudes
plt.subplot(2, 3, 1)
spam_lengths = df[df['category'] == 'Spam']['length']
no_spam_lengths = df[df['category'] == 'No Spam']['length']

plt.hist(spam_lengths, alpha=0.7, label='Spam', bins=10, color='red')
plt.hist(no_spam_lengths, alpha=0.7, label='No Spam', bins=10, color='blue')
plt.xlabel('Longitud del Email')
plt.ylabel('Frecuencia')
plt.legend()
plt.title('Distribución de Longitud por Categoría')

# Subplot 2: Análisis de tiendas mencionadas
plt.subplot(2, 3, 2)
stores = ['Amazon', 'eBay', 'AliExpress', 'TEMU']
store_counts = {}

for store in stores:
    store_counts[store] = sum(1 for email in emails if store.lower() in email.lower())

plt.bar(store_counts.keys(), store_counts.values(), color=['orange', 'yellow', 'red', 'purple'])
plt.title('Frecuencia de Tiendas Mencionadas')
plt.ylabel('Número de Menciones')
plt.xticks(rotation=45)

# Subplot 3: Palabras clave de spam
plt.subplot(2, 3, 3)
spam_keywords = ['OFERTA', 'EXCLUSIVO', 'GRATIS', 'DESCUENTO', 'URGENTE', 'GANASTE', 'PREMIO', 'FLASH']
keyword_counts = {}

spam_emails = df[df['category'] == 'Spam']['email']
for keyword in spam_keywords:
    keyword_counts[keyword] = sum(1 for email in spam_emails if keyword.lower() in email.lower())

plt.barh(list(keyword_counts.keys()), list(keyword_counts.values()), color='lightcoral')
plt.title('Palabras Clave en Emails Spam')
plt.xlabel('Frecuencia')

# Subplot 4: Análisis de signos de exclamación
plt.subplot(2, 3, 4)
df['exclamation_count'] = df['email'].str.count('!')
exclamation_by_category = df.groupby('category')['exclamation_count'].mean()

plt.bar(exclamation_by_category.index, exclamation_by_category.values, color=['lightblue', 'lightcoral'])
plt.title('Promedio de Signos de Exclamación')
plt.ylabel('Promedio por Email')

# Subplot 5: Análisis de números/precios
plt.subplot(2, 3, 5)
df['has_numbers'] = df['email'].str.contains(r'\d+', regex=True)
number_analysis = df.groupby('category')['has_numbers'].mean() * 100

plt.bar(number_analysis.index, number_analysis.values, color=['lightgreen', 'salmon'])
plt.title('Porcentaje de Emails con Números')
plt.ylabel('Porcentaje (%)')

# Subplot 6: Análisis de urgencia
plt.subplot(2, 3, 6)
urgency_words = ['URGENTE', 'AHORA', 'YA', 'INMEDIATAMENTE', 'ÚLTIMO', 'EXPIRA']
df['urgency_score'] = df['email'].apply(lambda x: sum(1 for word in urgency_words if word.lower() in x.lower()))
urgency_by_category = df.groupby('category')['urgency_score'].mean()

plt.bar(urgency_by_category.index, urgency_by_category.values, color=['lightcyan', 'orange'])
plt.title('Promedio de Palabras de Urgencia')
plt.ylabel('Promedio por Email')

plt.tight_layout()
plt.show()

#Plot 7 nube de palabras en emails de tipo Spam

# Unir todos los emails spam en un solo texto
spam_text = ' '.join(spam_emails)

# Crear la nube de palabras
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    colormap='Reds',
    max_words=50,
    relative_scaling=0.5,
    stopwords={'de', 'en', 'el', 'la', 'y', 'a', 'con', 'para', 'tu', 'ha', 'te', 'si', 'o'}
).generate(spam_text)

# Mostrar la nube de palabras
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras - Emails SPAM', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Estadísticas adicionales
print("Análisis Estadístico Adicional")
print("---------------------------")
print(f"Longitud promedio - Spam: {spam_lengths.mean():.1f} caracteres")
print(f"Longitud promedio - No Spam: {no_spam_lengths.mean():.1f} caracteres")
print(f"Tienda más mencionada: {max(store_counts, key=store_counts.get)} ({max(store_counts.values())} veces)")
print(f"Palabra clave más común en spam: {max(keyword_counts, key=keyword_counts.get)} ({max(keyword_counts.values())} veces)")

In [ ]:
# Sistema de reglas mejorado
class SpamRuleEngine:
    def __init__(self):
        # Palabras clave específicas para tiendas virtuales fraudulentas
        self.spam_keywords = {
            'aliexpress': ['aliexpress', 'ali express', 'ali-express'],
            'amazon': ['amazon', 'amazon prime', 'prime day'],
            'ebay': ['ebay', 'e-bay', 'e bay'],
            'temu': ['temu'],
            'urgency': ['urgente', 'último día', 'oferta limitada', 'solo hoy', 'expira'],
            'money': ['gratis', 'descuento', 'premio', '$', 'reembolso', 'ganaste'],
            'action': ['haz clic', 'reclama', 'verifica', 'confirma', 'compra ya']
        }

        self.legitimate_patterns = [
            'tu pedido', 'confirmación', 'número de seguimiento',
            'ha sido enviado', 'entregado exitosamente', 'reunión'
        ]

    def calculate_spam_score(self, text):
        text_lower = text.lower()
        score = 0

        # Puntuación por tiendas virtuales en contexto sospechoso
        for store, keywords in self.spam_keywords.items():
            if store in ['aliexpress', 'amazon', 'ebay', 'temu']:
                for keyword in keywords:
                    if keyword in text_lower:
                        # Mayor puntuación si aparece con palabras de urgencia o dinero
                        if any(urgent in text_lower for urgent in self.spam_keywords['urgency']):
                            score += 3
                        elif any(money in text_lower for money in self.spam_keywords['money']):
                            score += 2
                        else:
                            score += 1

        # Puntuación por patrones de urgencia
        for urgent_word in self.spam_keywords['urgency']:
            if urgent_word in text_lower:
                score += 1

        # Puntuación por ofertas monetarias
        for money_word in self.spam_keywords['money']:
            if money_word in text_lower:
                score += 1

        # Reducir puntuación para patrones legítimos
        for legit_pattern in self.legitimate_patterns:
            if legit_pattern in text_lower:
                score -= 2

        return max(0, score)  # No permitir puntuaciones negativas

# Crear instancia del motor de reglas
rule_engine = SpamRuleEngine()

In [ ]:
# Procesamiento avanzado
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Remover caracteres especiales pero mantener espacios
    text = re.sub(r'[^a-záéíóúñ\s]', ' ', text)
    # Remover espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Aplicar preprocesamiento
processed_emails = [preprocess_text(email) for email in emails]

# Vectorización mejorada
# Usar TF-IDF en lugar de CountVectorizer para mejor rendimiento
vectorizer = TfidfVectorizer(
    max_features=1000,
    stop_words=None,  # Mantener todas las palabras para detectar spam
    ngram_range=(1, 2),  # Usar unigramas y bigramas
    min_df=1,
    max_df=0.95
)

X = vectorizer.fit_transform(processed_emails)
y = np.array(labels)

print(f"Forma de la matriz de características: {X.shape}")
print(f"Número de características extraídas: {X.shape[1]}")

In [ ]:
# división de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Datos de entrenamiento: {X_train.shape[0]} emails")
print(f"Datos de prueba: {X_test.shape[0]} emails")

# Entrenamiento
# Perceptrón con parámetros optimizados
model = Perceptron(
    max_iter=1000,
    random_state=42,
    eta0=0.1,  # Tasa de aprendizaje
    penalty='l2',  # Regularización
    alpha=0.0001
)

model.fit(X_train, y_train)
print("✅ Modelo entrenado exitosamente!")

In [ ]:
# Guardar Modelos
joblib.dump(model, 'spam_model_improved.pkl')
joblib.dump(vectorizer, 'vectorizer_improved.pkl')
joblib.dump(rule_engine, 'rule_engine.pkl')

print("✅ Modelos guardados exitosamente!")

In [ ]:
# Detección de Spam
def advanced_spam_detector(email_text):
    # Cargar modelos
    model = joblib.load('spam_model_improved.pkl')
    vectorizer = joblib.load('vectorizer_improved.pkl')
    rule_engine = joblib.load('rule_engine.pkl')

    # Preprocesar texto
    processed_text = preprocess_text(email_text)

    # Vectorizar
    email_vector = vectorizer.transform([processed_text])

    # Predicción del perceptrón
    ml_prediction = model.predict(email_vector)[0]
    ml_confidence = model.decision_function(email_vector)[0]

    # Puntuación de reglas
    rule_score = rule_engine.calculate_spam_score(email_text)

    # Decisión final combinada
    if ml_prediction == 1 and rule_score >= 3:
        final_prediction = "SPAM (Alta confianza)"
        confidence = "Alta"
    elif ml_prediction == 1 or rule_score >= 5:
        final_prediction = "SPAM (Confianza media)"
        confidence = "Media"
    elif rule_score >= 2:
        final_prediction = "Posible SPAM (Baja confianza)"
        confidence = "Baja"
    else:
        final_prediction = "NO SPAM"
        confidence = "Alta"

    # Análisis detallado
    analysis = f"""
**ANÁLISIS DE SPAM**

**Resultado:** {final_prediction}
**Confianza:** {confidence}

**Detalles técnicos:**
• Predicción ML: {'SPAM' if ml_prediction == 1 else 'NO SPAM'}
• Puntuación de reglas: {rule_score}/10
• Confianza ML: {ml_confidence:.3f}

**Análisis de contenido:**
• Longitud del texto: {len(email_text)} caracteres
• Palabras clave detectadas: {rule_score > 0}
• Tiendas virtuales mencionadas: {any(store in email_text.lower() for store in ['aliexpress', 'amazon', 'ebay', 'temu'])}

**Recomendación:** {'No abrir enlaces ni proporcionar información personal' if 'SPAM' in final_prediction else 'Email parece legítimo'}
"""

    return analysis

In [ ]:
# Test 
print("🧪 Ejecutando pruebas automáticas...")

test_emails = [
    "¡URGENTE! AliExpress cierra con 99% descuento. Compra ya!",
    "Amazon: problema con tu cuenta, verifica inmediatamente",
    "Hola, ¿cómo va el proyecto? Nos vemos en la reunión",
    "eBay: Tu artículo favorito tiene 95% descuento. Solo hoy!",
    "TEMU: Confirmación de entrega de tu pedido #12345"
]

print("\nResultados de pruebas:")
for i, email in enumerate(test_emails, 1):
    result = advanced_spam_detector(email)
    # Extraer solo el resultado principal
    newline = '\n'
    resultado_linea = result.split('**Resultado:**')[1].split(newline)[0].strip()
    print(f"\nPrueba {i}: {email[:50]}...")
    print(f"Resultado: {resultado_linea}")

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import time
from pyngrok import ngrok
import re

app = Flask(__name__)
CORS(app) 

# https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("1uKgnupvgswLCNcPpobpadMACbE_7gw3hNMGf41eF8CjAndSt")

# Función para convertir markdown a JSON
def markdown_to_json(markdown_result, email_text):
    """Convierte el resultado markdown de advanced_spam_detector a JSON"""
    try:
        lines = markdown_result.split('\n')

        # Extraer información principal
        resultado_line = [line for line in lines if '**Resultado:**' in line][0]
        confianza_line = [line for line in lines if '**Confianza:**' in line][0]

        prediction = resultado_line.split('**Resultado:**')[1].strip()
        confidence = confianza_line.split('**Confianza:**')[1].strip()

        # Determinar si es spam
        is_spam = 'SPAM' in prediction.upper()

        # Extraer detalles técnicos
        ml_pred_line = [line for line in lines if '• Predicción ML:' in line][0]
        rule_score_line = [line for line in lines if '• Puntuación de reglas:' in line][0]

        ml_prediction = ml_pred_line.split('• Predicción ML:')[1].strip()
        rule_score = int(rule_score_line.split('• Puntuación de reglas:')[1].split('/')[0].strip())

        # Análisis de tiendas virtuales
        stores_mentioned = []
        for store in ['aliexpress', 'amazon', 'ebay', 'temu']:
            if store in email_text.lower():
                stores_mentioned.append(store.title())

        # Crear respuesta JSON estructurada
        response = {
            "status": "success",
            "result": {
                "is_spam": is_spam,
                "prediction": prediction,
                "confidence": confidence,
                "details": {
                    "ml_prediction": ml_prediction,
                    "rule_score": rule_score,
                    "rule_score_max": 10
                },
                "content_analysis": {
                    "text_length": len(email_text),
                    "virtual_stores_mentioned": stores_mentioned,
                    "stores_count": len(stores_mentioned)
                },
                "recommendation": "No abrir enlaces ni proporcionar información personal" if is_spam else "Email parece legítimo",
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
            }
        }

        return response

    except Exception as e:
        return {
            "status": "error",
            "message": f"Error procesando resultado: {str(e)}",
            "raw_result": markdown_result
        }

@app.route('/detect', methods=['POST'])
def detect_spam():
    """Endpoint principal para detectar spam - USA advanced_spam_detector"""
    try:
        # Obtener datos del request
        data = request.get_json()

        # Validar que se envió el texto del email
        if not data or 'email_text' not in data:
            return jsonify({
                "status": "error",
                "message": "Se requiere el campo 'email_text' en el body del request",
                "example": {
                    "email_text": "¡Oferta exclusiva! Compra ahora en Amazon..."
                }
            }), 400

        email_text = data['email_text']

        # Validar que el texto no esté vacío
        if not email_text or len(email_text.strip()) == 0:
            return jsonify({
                "status": "error",
                "message": "El texto del email no puede estar vacío"
            }), 400

        # 🎯 USAR LA FUNCIÓN EXISTENTE advanced_spam_detector
        markdown_result = advanced_spam_detector(email_text)

        # Convertir resultado markdown a JSON
        json_result = markdown_to_json(markdown_result, email_text)

        return jsonify(json_result)

    except Exception as e:
        return jsonify({
            "status": "error",
            "message": f"Error interno del servidor: {str(e)}"
        }), 500

def run_flask_app():
    """Función para ejecutar Flask en un hilo separado"""
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Iniciar Flask en un hilo separado
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

time.sleep(3)

# Configurar ngrok para acceso público
try:
    public_url = ngrok.connect(5000)
    print(f"URL pública: {public_url}")

except Exception as e:
    print(f"⚠️ Error configurando ngrok: {e}")